In [1]:
import pandas as pd
from thefuzz import fuzz

In [2]:
divs = pd.read_csv('section_boundaries.csv', index_col=0)

In [3]:
divs = divs[divs.start_phrase.notna()]

In [4]:
divs

,datecode,type,start_phrase,end_phrase,Unnamed: 5,Unnamed: 6
208,19990203,FOMC2,"Thank you, Mr. Chairman. I will begin my brief...",This language begins at the bottom of page 23 ...,NaN,NaN
209,19990203,FOMC1,"Thank you, Mr. Chairman. I will start the pres...","Thank you. Mr. Chairman, I would not have thou...",NaN,NaN
210,19990330,FOMC2,I will be referring to the table marked “Finan...,"I will begin with that, Mr. Chairman. There is...",NaN,NaN
211,19990330,FOMC1,"Thank you, Mr. Chairman. For the benefit of th...","Thank you, Mr. Chairman. In the material we re...",NaN,NaN
212,19990518,FOMC2,"Thank you, Mr. Chairman. At your last meeting,...",I am reading from page 13 of the Bluebook: “To...,NaN,NaN
...,...,...,...,...,...,...
509,20170920,FOMC1,All in favor? [Chorus of ayes] Any opposed? [N...,"Well, my thanks to everyone for a rich discuss...",NaN,NaN
510,20171101,FOMC1,Thank you. All in favor? [Chorus of ayes] Any ...,Thank you. My thanks to everyone for a thought...,NaN,NaN
511,20171101,FOMC2,"Good morning, everybody. Let’s get started. An...","Thank you, Madam Chair. As you indicated, this...",NaN,NaN
512,20171213,FOMC1,Thank you. All in favor? [Chorus of ayes] I’m ...,"Thank you. Well, my thanks to everyone for a t...",NaN,NaN


In [5]:
new = divs.pivot(index='datecode', columns='type', values=['start_phrase', 'end_phrase']).reset_index()
new

datecode                                       start_phrase  \
type                                                        FOMC1   
0     19990203  Thank you, Mr. Chairman. I will start the pres...   
1     19990330  Thank you, Mr. Chairman. For the benefit of th...   
2     19990518  Thank you, Mr. Chairman. The April Consumer Pr...   
3     19990630  I don’t know how real the economics are, but t...   
4     19990824  Thank you, Mr. Chairman. At first blush, it mi...   
..         ...                                                ...   
147   20170614  That’s right. Okay. Seeing no further comments...   
148   20170726  Thank you. All in favor. [Chorus of ayes] Than...   
149   20170920  All in favor? [Chorus of ayes] Any opposed? [N...   
150   20171101  Thank you. All in favor? [Chorus of ayes] Any ...   
151   20171213  Thank you. All in favor? [Chorus of ayes] I’m ...   

                                                         \
type                                              FOMC2   
0     Thank you, Mr. Chairman. I will begin my brief...   
1     I will be referring to the table marked “Finan...   
2     Thank you, Mr. Chairman. At your last meeting,...   
3     Thank you, Mr. Chairman. The long-run strategi...   
4     Thank you, Mr. Chairman. Judging from the rema...   
..                                                  ...   
147   Thank you, Madam Chair. I will be referring to...   
148   Thank you, Madam Chair. I will be referring to...   
149   Thank you, Madam Chair. I will be referring to...   
150   Good morning, everybody. Let’s get started. An...   
151   Any questions for David? [No response] Okay. S...   

                                             end_phrase  \
type                                              FOMC1   
0     Thank you. Mr. Chairman, I would not have thou...   
1     Thank you, Mr. Chairman. In the material we re...   
2     I was quoted in a recent New Yorker article, o...   
3     Thank you, Mr. Chairman. We’re being increasin...   
4     Thank you, Mr. Chairman. I believe I am the fi...   
..                                                  ...   
147   Thank you. And thanks, everyone, for an insigh...   
148   Thank you. Thanks to everyone for a thoughtful...   
149   Well, my thanks to everyone for a rich discuss...   
150   Thank you. My thanks to everyone for a thought...   
151   Thank you. Well, my thanks to everyone for a t...   

                                                         
type                                              FOMC2  
0     This language begins at the bottom of page 23 ...  
1     I will begin with that, Mr. Chairman. There is...  
2     I am reading from page 13 of the Bluebook: “To...  
3     This time we’re on page 22 of the Bluebook, at...  
4     I’m reading from page 15 of the Bluebook: “To ...  
..                                                  ...  
147   Thank you, Madam Chair. As you noted, this vot...  
148   Thank you, Madam Chair. This vote will be on t...  
149   Thank you, Madam Chair. As you noted, this vot...  
150   Thank you, Madam Chair. As you indicated, this...  
151   Thank you, Madam Chair. The vote will be on th...  

[152 rows x 5 columns]

In [6]:
rt = pd.read_csv('../working-csvs/raw_transcripts.csv', index_col=0)
rt = rt.dropna(subset=['content'])

In [7]:
res_dfs = []
for group in rt.groupby('date'):
    datecode = int(group[0].replace('-', '')) 
    if datecode not in divs['datecode'].unique():
        continue
    FOMC1_bounds = [new.loc[new.datecode == datecode].start_phrase.FOMC1.iloc[0], new.loc[new.datecode == datecode].end_phrase.FOMC1.iloc[0]]
    FOMC2_bounds = [new.loc[new.datecode == datecode].start_phrase.FOMC2.iloc[0], new.loc[new.datecode == datecode].end_phrase.FOMC2.iloc[0]]
    sections = [FOMC1_bounds, FOMC2_bounds]
    transc = group[1]
    for i, section in enumerate(sections):
        transc[f'sec{i + 1}startsim'] = transc['content'].apply(lambda x : fuzz.ratio(x[:len(section[0]) + 10], section[0]))
        transc[f'sec{i + 1}endsim'] = transc['content'].apply(lambda x : fuzz.ratio(x[:len(section[1]) + 10], section[1]))
    maxes = transc.idxmax(numeric_only=True)
    FOMC1 = transc.loc[maxes.sec1startsim:maxes.sec1endsim].copy()
    FOMC1['section'] = 1
    FOMC2 = transc.loc[maxes.sec2startsim:maxes.sec2endsim].copy()
    FOMC2['section'] = 2
    result = pd.concat([FOMC1, FOMC2])
    result = result.loc[:, ~result.columns.isin(['sec1startsim', 'sec1endsim', 'sec2startsim', 'sec2endsim'])]
    res_dfs.append(result)

In [9]:
sectioned = pd.concat(res_dfs)

In [10]:
sectioned.to_csv('../working-csvs/raw_transcripts_sectioned.csv')